# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Carrera: <Coloca el nombre de tu carrera aqui>** </center>
---
### <center> **Primavera 2025** </center>
---

**Lab 11**: Multi classification (Decision Trees & SVM)

**Fecha**: 10/05/2025

**Nombre del Estudiante**:
Jose Cota
Andres Blanco
Jose Hurtado
Jose Mendoza

**Profesor**: Pablo Camarillo Ramirez